In [2]:
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import pandas

In [4]:
print("Getting list of topic names and texts...")
with open("01_topic_text.csv") as file:
    topic_names = []
    topic_texts = []
    for line in csv.reader(file):
        topic_names.append(line[0])
        topic_texts.append(line[1])
print("Done!")

Getting list of topic names and texts...
Done!


In [5]:
topic_id = 1
print(f"Topic {topic_id} is:", topic_names[topic_id])
print(topic_texts[topic_id])

Topic 1 is: configuring-project-and-ide-settings.html
Configuring the IDE
IntelliJ IDEA allows you to configure the settings on several levels: the module level, the project level, and globally.
Global settings apply to all projects that you open with a specific installation or version of IntelliJ IDEA. Such settings include IDE appearance (themes, color schemes, menus and toolbars), notification settings, the set of the installed and enabled plugins, debugger settings, code completion, and so on.
To configure your IDE, select IntelliJ IDEA | Preferences for macOS or File | Settings for Windows and Linux. Alternatively, press Ctrl+Alt+S or click  on the toolbar.
Settings that are NOT marked with the  icon in the Settings/Preferences dialog are global and apply to all existing projects of the current IntelliJ IDEA version.
Restore IDE settings
In IntelliJ IDEA 2020.1, the configuration paths have been changed. That is why, the information in this section is valid for IntelliJ IDEA of ve

In [6]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
corpus_embeddings = embedder.encode(topic_texts, convert_to_tensor=True)

In [29]:
query = "file templates"
query_embedding = embedder.encode(query, convert_to_tensor=True)

In [30]:
cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=5)

print("Query:", query)
print("\nTop 5 most similar docs in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    print(topic_names[idx], "(Score: {:.4f})".format(score))

Query: file templates

Top 5 most similar docs in corpus:
using-file-and-code-templates.html (Score: 0.6213)
settings-file-and-code-templates.html (Score: 0.5908)
templates-with-multiple-files.html (Score: 0.5699)
parse-directive.html (Score: 0.5674)
template-data-languages.html (Score: 0.5546)


In [15]:
results_csv = "search_test_results.csv"

with open(results_csv, "r") as file:
    df = pandas.read_csv(file)

search_queries = df["query"].tolist()

all_results = []

for query in search_queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=5)

    q_result = ""

    for score, idx in zip(top_results[0], top_results[1]):
        q_result = q_result + topic_names[idx] + "\n"

    all_results.append(q_result)

# print(all_results)
df["sbert"] = all_results

In [16]:
df.to_csv(results_csv, index=False)